# LRZ Linux Cluster: Getting Started with Pilot-Streaming 

In the first step we need to import all required packages and modules into the Python Path

In [ ]:
# System Libraries
import sys, os
sys.path.append("..")
import pandas as pd

## logging
import logging
logging.basicConfig(level=logging.DEBUG)
logging.getLogger().setLevel(logging.ERROR)
logging.getLogger("py4j").setLevel(logging.ERROR)
 

# Pilot-Streaming
import pilot.streaming
sys.modules['pilot.streaming']

The Pilot-Compute Description is a simple key/value style description of the cluster environment that should be started. Alternatively, the commandline tool delivered with this package can be used:

     pilot-streaming --resource=slurm://localhost --queue=normal --walltime=59 --number_cores=48 --framework spark 

# 1. Spark

In [ ]:
### Required Spark configuration that needs to be provided before pyspark is imported and JVM started
#os.environ["SPARK_LOCAL_IP"]='129.114.58.2' #must be done before pyspark is loaded
import pyspark

pilot_compute_description = {
    "resource":"slurm+ssh://mpp2-login6",
    "working_directory": os.path.join('/home/hpc/ug201/di57hah/project/', "work"),
    "number_cores": 56,
    "walltime": 59,
    "type":"spark"
}

Start Spark Cluster and Wait for Startup Completion

In [ ]:
%%time

spark_pilot = pilot.streaming.PilotComputeService.create_pilot(pilot_compute_description)
spark_pilot.wait()

In [ ]:
spark_pilot.get_details()

In [ ]:
#sc = pyspark.SparkContext(master="spark://129.114.58.135:7077", appName="test")

In [ ]:
sc = spark_pilot.get_context()

In [ ]:
rdd = sc.parallelize([1,2,3])
rdd.map(lambda a: a*a).collect()

In [ ]:
spark_pilot.cancel()

# 2. Kafka

In [ ]:
pilot_compute_description = {
    "resource":"slurm://localhost",
    "working_directory": os.path.join('/work/01131/tg804093/wrangler/', "work"),
    "number_cores": 48,
    "project": "TG-MCB090174",
    "queue": "normal",
    "walltime": 59,
    "type":"kafka"
}

In [ ]:
%%time
kafka_pilot = pilot.streaming.PilotComputeService.create_pilot(pilot_compute_description)
kafka_pilot.wait()

In [ ]:
kafka_pilot.get_details()

In [ ]:
kafka_pilot.cancel()

# 3. Dask

In [ ]:
import distributed

pilot_compute_description = {
    "resource":"slurm://localhost",
    "working_directory": os.path.join('/work/01131/tg804093/wrangler/', "work"),
    "number_cores": 48,
    "project": "TG-MCB090174",
    "queue": "normal",
    "walltime": 59,
    "type":"dask"
}

In [ ]:
%%time
dask_pilot = pilot.streaming.PilotComputeService.create_pilot(pilot_compute_description)
dask_pilot.wait()

In [ ]:
dask_pilot.get_details()

In [ ]:
import distributed
dask_client  = distributed.Client(dask_pilot.get_details()['master_url'])
dask_client.scheduler_info()

In [ ]:
dask_client.gather(dask_client.map(lambda a: a*a, range(10)))